In [1]:
import pandas as pd 
import numpy as np

from tqdm import tqdm
# from fasttext import load_model
import gensim
from pymorphy2 import MorphAnalyzer

In [2]:
morph = MorphAnalyzer()

In [3]:
# !pip install pymorphy2

In [4]:
X_train = pd.read_csv('train_stat.csv')
X_test = pd.read_csv('test_stat.csv')

In [5]:
X_train.shape

(283086, 27)

In [6]:
def vector(q, word2vec_model):
    w = morph.parse(q)[0]
    if w.tag.POS in mapping:
        new_w = '{}{}'.format(w.normal_form, mapping[w.tag.POS])
    
        qf = q

        if q not in word2vec_model:
            candidates_set = set()

            candidates_set.add(q.upper())
            candidates_set.add(q.lower())
            candidates_set.add(q.capitalize())

            no_results = True
            for candidate in candidates_set:
                if candidate in word2vec_model:
                    qf = candidate
                    no_results = False
                    break

            if no_results:
                # obvious that not all elements
                # in corpus will from our vocabulary
                return np.zeros(300)

        raw_vector = word2vec_model[qf]
        return raw_vector
    return np.zeros(300)

In [7]:
def get_word2vec_model(binary_filename, **kwargs):
    model = gensim.models.KeyedVectors.load_word2vec_format(
        binary_filename,
#         binary=kwargs.get('binary')
    )

    model.init_sims(replace=True)
    return model

In [8]:
model = get_word2vec_model('ruwikiruscorpora-superbigrams_skipgram_300_2_2018.vec.gz', binary='.vec.gz')

In [11]:
def sent2vec(words, model):
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(vector(w, model).astype('float32'))
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [12]:
X_train

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,...,tfidf_char_nb_eap_аллергия,tfidf_char_nb_eap_Анамнез заболевания,tfidf_char_nb_eap_Внешний осмотр,count_words_nb_eap_Диагноз,count_words_nb_eap_Общее состояние,count_words_nb_eap_аллергия,count_words_nb_eap_Анамнез заболевания,count_words_nb_eap_Внешний осмотр,Код_диагноза_0,Код_диагноза_1
0,0,1327,"['острый', 'ларингофарингит']",29,2,[],[],[],"['на', 'жевательный', 'поверхность', '2.6-', '...",1,...,1.765447e-01,0.014373,4.831771e-08,4.227677e-02,1.707658e-01,0.170766,1.707658e-01,1.884889e-15,10,6
1,1,2469,"['подострый', 'хронический', 'вагинит']",45,2,[],[],"['считать', 'больной', 'протяжение', 'многий',...",[],1,...,1.801849e-01,0.000255,1.944706e-01,1.998505e-03,1.718646e-01,0.171865,3.496850e-17,1.718646e-01,14,76
2,2,1701,"['аллергический', 'контактный', 'дерматит,', '...",63,2,[],[],"['больной', 'около', '2-х', 'дней,', 'появитьс...",[],1,...,1.774575e-01,0.000200,1.927694e-01,8.238913e-03,1.711360e-01,0.171136,5.596109e-09,1.711360e-01,12,23
3,3,2408,['баланопостить'],43,2,[],['отяготить'],[],[],1,...,1.695790e-04,0.014373,1.914799e-01,2.088008e-01,1.707658e-01,0.063228,1.707658e-01,1.707658e-01,14,48
4,4,3324,"['общий', 'медицинский', 'осмотр']",29,1,[],[],"['провести', 'полный', 'обследование', 'операт...",[],1,...,1.765447e-01,0.000749,1.914799e-01,8.554433e-01,1.707658e-01,0.170766,4.949576e-04,1.707658e-01,25,0
5,5,3330,"['другой', 'специальный', 'осмотр', 'обследова...",50,2,['удовлетворительное.'],['отяготить'],[],[],1,...,1.968386e-04,0.014819,1.927694e-01,9.892252e-01,1.460410e-01,0.065309,1.711360e-01,1.711360e-01,25,1
6,6,1987,"['поражение', 'мениск', 'результат', 'старое',...",42,2,[],[],"['проводить', 'лечение,', 'прибыть', 'коррекци...",[],1,...,1.801849e-01,0.000616,1.944706e-01,9.999959e-01,1.718646e-01,0.171865,1.444603e-04,1.718646e-01,13,23
7,7,446,"['тиреотоксикоз', 'диффузный', 'зоб']",85,1,[],[],"['вышеописанный', 'жалоба', 'беспокоить', '03....",[],0,...,1.765447e-01,0.000186,1.914799e-01,1.254906e-01,1.707658e-01,0.170766,3.725213e-03,1.707658e-01,5,5
8,8,2476,"['вагинит,', 'вульвит', 'вульвовагинит', 'инфе...",28,2,"['общий', 'самочувствие', 'удовлетворительное....",[],[],[],1,...,1.801849e-01,0.014366,1.944706e-01,2.541414e-14,3.543579e-06,0.171865,1.718646e-01,1.718646e-01,14,77
9,9,996,"['наружный', 'отит']",38,1,[],[],"['ранее', 'эндокринолог', 'обращаться', 'обсле...",[],1,...,1.780040e-01,0.000845,1.936453e-01,2.876322e-01,1.712729e-01,0.171273,1.294402e-03,1.712729e-01,8,60


In [13]:
def doGlove(x_train,x_test):
    # create sentence vectors using the above function for training and validation set
    xtrain_glove = [sent2vec(x, model) for x in tqdm(x_train)]
    xtest_glove = [sent2vec(x, model) for x in tqdm(x_test)]
    xtrain_glove = np.array(xtrain_glove)
    xtest_glove = np.array(xtest_glove)
    return xtrain_glove,xtest_glove


In [14]:
for f in ['Диагноз', 'Общее состояние', 'аллергия', 'Анамнез заболевания', 'Внешний осмотр']:
    glove_vecs_train, glove_vecs_test = doGlove(X_train[f],X_test[f])
    X_train[['sent_vec_{}_{}'.format(f, str(i)) for i in range(300)]] = pd.DataFrame(glove_vecs_train.tolist())
    X_test[['sent_vec_{}_{}'.format(f, str(i)) for i in range(300)]] = pd.DataFrame(glove_vecs_test.tolist())

100%|██████████| 100000/100000 [07:20<00:00, 227.11it/s]


In [15]:
X_train.to_csv('train_cool_glove.csv')
X_test.to_csv('test_cool_glove.csv')

In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283086 entries, 0 to 283085
Columns: 1527 entries, ID to sent_vec_Внешний осмотр_299
dtypes: float64(1515), int64(7), object(5)
memory usage: 3.2+ GB


In [19]:
X_train.columns[:50]

Index(['ID', 'Код_диагноза', 'Диагноз', 'Возраст', 'Пол', 'Общее состояние',
       'аллергия', 'Анамнез заболевания', 'Внешний осмотр', 'Revisit',
       'tfidf_nb_eap_tfidfAnamnes', 'tfidf_nb_eap_tfidfDiagnoz',
       'tfidf_nb_eap_tfidfGeneral', 'tfidf_nb_eap_tfidfAllergy',
       'tfidf_nb_eap_tfidfOutside', 'tfidf_char_nb_eap_Диагноз',
       'tfidf_char_nb_eap_Общее состояние', 'tfidf_char_nb_eap_аллергия',
       'tfidf_char_nb_eap_Анамнез заболевания',
       'tfidf_char_nb_eap_Внешний осмотр', 'count_words_nb_eap_Диагноз',
       'count_words_nb_eap_Общее состояние', 'count_words_nb_eap_аллергия',
       'count_words_nb_eap_Анамнез заболевания',
       'count_words_nb_eap_Внешний осмотр', 'Код_диагноза_0', 'Код_диагноза_1',
       'sent_vec_Диагноз_0', 'sent_vec_Диагноз_1', 'sent_vec_Диагноз_2',
       'sent_vec_Диагноз_3', 'sent_vec_Диагноз_4', 'sent_vec_Диагноз_5',
       'sent_vec_Диагноз_6', 'sent_vec_Диагноз_7', 'sent_vec_Диагноз_8',
       'sent_vec_Диагноз_9', 'sent_vec